In [8]:
import tensorflow as tf
import numpy as np

#输入图像尺寸
image_width = 10
image_height = 10
image_depth = 3

#卷积数量=输出层的深度
k_output = 64
#convolution fliter卷积核或者过滤器的尺寸
filter_width = 5
filter_height = 5

#输入
inputs = tf.placeholder(tf.float32,[None,image_width,image_height,image_depth])

#weights and biasis
weights = tf.Variable(tf.truncated_normal([filter_width,filter_height,image_depth,k_output]))
biasis = tf.Variable(tf.zeros(k_output))

#apply convolution
"""
inputs的每一个维度的步长都单独设置
strides = [batch, input_height, input_width, input_channels],一般将batch和input_channels设为1
padding = 'same'或者'valid' same就代表扩展padding，补充0，使得在步长为1时输出图像尺寸与输入相同
"""
layer_1 = tf.nn.conv2d(inputs,weights,strides=[1,2,2,1],padding = 'SAME')
layer_2 = tf.nn.bias_add(layer_1,biasis)
#使用激励函数处理输出
output_layer = tf.nn.relu(layer_2)

#使用max_pooling最大池化,ksize是滤波器的大小
output_layer = tf.nn.max_pool(output_layer,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')


### 下面使用tensorflow构建关于mnist数据集的简单卷积神经网络

In [15]:
#Mnist数据集，使用tensorflow & cnn
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

#设置参数
learning_rate = 0.00001
epochs =  10
batch_size = 128 

# 用来验证和计算准确率的样本数
# 如果内存不够，可以调小这个数字
test_valid_size = 256

#神经网络参数
n_classes = 10
dropout = 0.75

#weights and biasis
#filter_width,filter_height,filter_depth,filter_nums(k_output)
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}  

biasis={
    'bc1':tf.Variable(tf.random_normal([32])),
    'bc2':tf.Variable(tf.random_normal([64])),
    'bd1':tf.Variable(tf.random_normal([1024])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}
#卷积层
def conv2d(x,w,b,strides=1):
    x = tf.nn.conv2d(x,w,strides=[1,strides,strides,1],padding = 'SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)
#池化层
def maxpool_2d(x,k=2):
    return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')

#神经网络构造
def conv_net(x,weights,biasis,dropout):
    conv1 = conv2d(x,weights['wc1'], biasis['bc1'])
    conv1 = maxpool_2d(conv1,k=2)
    
    conv2 = conv2d(conv1,weights['wc2'],biasis['bc2'])
    conv2 = maxpool_2d(conv2,k=2)
    
    #全连接层
    fc1 = tf.reshape(conv2,[-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1,weights['wd1']),biasis['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1,dropout)
    
    #输出
    output = tf.add(tf.matmul(fc1,weights['out']),biasis['out'])
    return output                      

    

Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz


In [23]:
#运行网络

x = tf.placeholder(tf.float32,[None,28,28,1])
y = tf.placeholder(tf.float32,[None,n_classes])
keep_prob = tf.placeholder(tf.float32)#这个应该是用于dropout的，用来去除数据的一部分，以防止过拟合

#model
logits = conv_net(x,weights,biasis,keep_prob)

#loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)

#accuracy
corrent_pred = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(corrent_pred,tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer,feed_dict={x:batch_x,y:batch_y,keep_prob:dropout})
            
            loss = sess.run(cost,feed_dict={x:batch_x,y:batch_y,keep_prob:1.})
            
            valid_acc = sess.run(accuracy,feed_dict = {x:mnist.validation.images[:test_valid_size],
                                                        y:mnist.validation.labels[:test_valid_size],
                                                        keep_prob:1.})
            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))
            
    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))



Epoch  1, Batch   1 -Loss: 51224.5547 Validation Accuracy: 0.128906
Epoch  1, Batch   2 -Loss: 49405.1836 Validation Accuracy: 0.136719
Epoch  1, Batch   3 -Loss: 35684.9961 Validation Accuracy: 0.132812
Epoch  1, Batch   4 -Loss: 29439.2891 Validation Accuracy: 0.132812
Epoch  1, Batch   5 -Loss: 27321.6797 Validation Accuracy: 0.144531
Epoch  1, Batch   6 -Loss: 26184.6562 Validation Accuracy: 0.171875
Epoch  1, Batch   7 -Loss: 27082.4160 Validation Accuracy: 0.171875
Epoch  1, Batch   8 -Loss: 22716.4492 Validation Accuracy: 0.207031
Epoch  1, Batch   9 -Loss: 22170.0215 Validation Accuracy: 0.175781
Epoch  1, Batch  10 -Loss: 19806.0820 Validation Accuracy: 0.207031
Epoch  1, Batch  11 -Loss: 17695.6855 Validation Accuracy: 0.242188
Epoch  1, Batch  12 -Loss: 20182.6289 Validation Accuracy: 0.207031
Epoch  1, Batch  13 -Loss: 18721.0176 Validation Accuracy: 0.218750
Epoch  1, Batch  14 -Loss: 17035.7695 Validation Accuracy: 0.257812
Epoch  1, Batch  15 -Loss: 18734.3828 Validation

Epoch  1, Batch 122 -Loss:  3363.2026 Validation Accuracy: 0.632812
Epoch  1, Batch 123 -Loss:  4214.7979 Validation Accuracy: 0.632812
Epoch  1, Batch 124 -Loss:  5065.3784 Validation Accuracy: 0.640625
Epoch  1, Batch 125 -Loss:  3581.2856 Validation Accuracy: 0.636719
Epoch  1, Batch 126 -Loss:  3686.8005 Validation Accuracy: 0.632812
Epoch  1, Batch 127 -Loss:  4225.1553 Validation Accuracy: 0.644531
Epoch  1, Batch 128 -Loss:  4224.3179 Validation Accuracy: 0.644531
Epoch  1, Batch 129 -Loss:  3742.8052 Validation Accuracy: 0.640625
Epoch  1, Batch 130 -Loss:  4962.9668 Validation Accuracy: 0.644531
Epoch  1, Batch 131 -Loss:  4473.8369 Validation Accuracy: 0.644531
Epoch  1, Batch 132 -Loss:  3382.8057 Validation Accuracy: 0.648438
Epoch  1, Batch 133 -Loss:  3914.8179 Validation Accuracy: 0.644531
Epoch  1, Batch 134 -Loss:  3416.0825 Validation Accuracy: 0.652344
Epoch  1, Batch 135 -Loss:  3986.6875 Validation Accuracy: 0.652344
Epoch  1, Batch 136 -Loss:  3944.8679 Validation

Epoch  1, Batch 243 -Loss:  2611.3281 Validation Accuracy: 0.757812
Epoch  1, Batch 244 -Loss:  2813.4050 Validation Accuracy: 0.757812
Epoch  1, Batch 245 -Loss:  1860.6940 Validation Accuracy: 0.757812
Epoch  1, Batch 246 -Loss:  2032.8582 Validation Accuracy: 0.765625
Epoch  1, Batch 247 -Loss:  2097.5918 Validation Accuracy: 0.750000
Epoch  1, Batch 248 -Loss:  2878.9556 Validation Accuracy: 0.761719
Epoch  1, Batch 249 -Loss:  2313.9324 Validation Accuracy: 0.753906
Epoch  1, Batch 250 -Loss:  2717.8938 Validation Accuracy: 0.750000
Epoch  1, Batch 251 -Loss:  2203.1504 Validation Accuracy: 0.753906
Epoch  1, Batch 252 -Loss:  2778.9758 Validation Accuracy: 0.761719
Epoch  1, Batch 253 -Loss:  1776.5684 Validation Accuracy: 0.757812
Epoch  1, Batch 254 -Loss:  2172.6445 Validation Accuracy: 0.753906
Epoch  1, Batch 255 -Loss:  2212.5386 Validation Accuracy: 0.750000
Epoch  1, Batch 256 -Loss:  2110.8838 Validation Accuracy: 0.753906
Epoch  1, Batch 257 -Loss:  1787.1786 Validation

Epoch  1, Batch 364 -Loss:  1488.4624 Validation Accuracy: 0.808594
Epoch  1, Batch 365 -Loss:  1633.0200 Validation Accuracy: 0.808594
Epoch  1, Batch 366 -Loss:  1790.0900 Validation Accuracy: 0.808594
Epoch  1, Batch 367 -Loss:  2321.6328 Validation Accuracy: 0.804688
Epoch  1, Batch 368 -Loss:  1981.9857 Validation Accuracy: 0.808594
Epoch  1, Batch 369 -Loss:  1313.5132 Validation Accuracy: 0.800781
Epoch  1, Batch 370 -Loss:  1309.9307 Validation Accuracy: 0.800781
Epoch  1, Batch 371 -Loss:  2022.5669 Validation Accuracy: 0.800781
Epoch  1, Batch 372 -Loss:  1950.5508 Validation Accuracy: 0.808594
Epoch  1, Batch 373 -Loss:  2588.1938 Validation Accuracy: 0.792969
Epoch  1, Batch 374 -Loss:  2129.6973 Validation Accuracy: 0.789062
Epoch  1, Batch 375 -Loss:  1453.7219 Validation Accuracy: 0.796875
Epoch  1, Batch 376 -Loss:  1579.9326 Validation Accuracy: 0.789062
Epoch  1, Batch 377 -Loss:  2144.8867 Validation Accuracy: 0.792969
Epoch  1, Batch 378 -Loss:  2073.9104 Validation

Epoch  2, Batch  56 -Loss:  1263.3601 Validation Accuracy: 0.800781
Epoch  2, Batch  57 -Loss:  1218.9751 Validation Accuracy: 0.804688
Epoch  2, Batch  58 -Loss:  1160.4666 Validation Accuracy: 0.804688
Epoch  2, Batch  59 -Loss:  1578.3926 Validation Accuracy: 0.804688
Epoch  2, Batch  60 -Loss:  1329.2000 Validation Accuracy: 0.808594
Epoch  2, Batch  61 -Loss:  1334.7683 Validation Accuracy: 0.812500
Epoch  2, Batch  62 -Loss:  1349.9204 Validation Accuracy: 0.812500
Epoch  2, Batch  63 -Loss:  1303.2706 Validation Accuracy: 0.812500
Epoch  2, Batch  64 -Loss:   788.4500 Validation Accuracy: 0.812500
Epoch  2, Batch  65 -Loss:  1414.1785 Validation Accuracy: 0.800781
Epoch  2, Batch  66 -Loss:   906.0105 Validation Accuracy: 0.796875
Epoch  2, Batch  67 -Loss:  1727.4824 Validation Accuracy: 0.796875
Epoch  2, Batch  68 -Loss:  1664.2938 Validation Accuracy: 0.796875
Epoch  2, Batch  69 -Loss:  1881.7234 Validation Accuracy: 0.800781
Epoch  2, Batch  70 -Loss:  1399.2561 Validation

...

Epoch  4, Batch 166 -Loss:   623.2625 Validation Accuracy: 0.871094
Epoch  4, Batch 167 -Loss:   622.4055 Validation Accuracy: 0.867188
Epoch  4, Batch 168 -Loss:   503.7223 Validation Accuracy: 0.867188
Epoch  4, Batch 169 -Loss:   597.9850 Validation Accuracy: 0.867188
Epoch  4, Batch 170 -Loss:   754.7769 Validation Accuracy: 0.867188
Epoch  4, Batch 171 -Loss:   608.9951 Validation Accuracy: 0.867188
Epoch  4, Batch 172 -Loss:   620.2875 Validation Accuracy: 0.867188
Epoch  4, Batch 173 -Loss:   515.4563 Validation Accuracy: 0.867188
Epoch  4, Batch 174 -Loss:   667.0381 Validation Accuracy: 0.863281
Epoch  4, Batch 175 -Loss:   585.4611 Validation Accuracy: 0.863281
Epoch  4, Batch 176 -Loss:   352.8898 Validation Accuracy: 0.863281
Epoch  4, Batch 177 -Loss:   691.5794 Validation Accuracy: 0.863281
Epoch  4, Batch 178 -Loss:   819.2047 Validation Accuracy: 0.863281
Epoch  4, Batch 179 -Loss:   632.8602 Validation Accuracy: 0.863281
Epoch  4, Batch 180 -Loss:   495.4390 Validation

Epoch  4, Batch 287 -Loss:   816.5182 Validation Accuracy: 0.878906
Epoch  4, Batch 288 -Loss:   577.5674 Validation Accuracy: 0.878906
Epoch  4, Batch 289 -Loss:   619.4662 Validation Accuracy: 0.871094
Epoch  4, Batch 290 -Loss:   534.1177 Validation Accuracy: 0.871094
Epoch  4, Batch 291 -Loss:   608.1334 Validation Accuracy: 0.875000
Epoch  4, Batch 292 -Loss:   661.8674 Validation Accuracy: 0.871094
Epoch  4, Batch 293 -Loss:   511.6657 Validation Accuracy: 0.878906
Epoch  4, Batch 294 -Loss:   609.2893 Validation Accuracy: 0.875000
Epoch  4, Batch 295 -Loss:   569.3451 Validation Accuracy: 0.878906
Epoch  4, Batch 296 -Loss:   453.6929 Validation Accuracy: 0.878906
Epoch  4, Batch 297 -Loss:   925.1216 Validation Accuracy: 0.871094
Epoch  4, Batch 298 -Loss:   333.8831 Validation Accuracy: 0.871094
Epoch  4, Batch 299 -Loss:   725.2891 Validation Accuracy: 0.875000
Epoch  4, Batch 300 -Loss:   626.9093 Validation Accuracy: 0.875000
Epoch  4, Batch 301 -Loss:   879.4597 Validation

Epoch  4, Batch 408 -Loss:   757.0266 Validation Accuracy: 0.871094
Epoch  4, Batch 409 -Loss:  1003.5008 Validation Accuracy: 0.871094
Epoch  4, Batch 410 -Loss:   433.7123 Validation Accuracy: 0.871094
Epoch  4, Batch 411 -Loss:   840.2131 Validation Accuracy: 0.871094
Epoch  4, Batch 412 -Loss:   785.4178 Validation Accuracy: 0.871094
Epoch  4, Batch 413 -Loss:   569.0946 Validation Accuracy: 0.871094
Epoch  4, Batch 414 -Loss:   425.1758 Validation Accuracy: 0.871094
Epoch  4, Batch 415 -Loss:   360.3726 Validation Accuracy: 0.867188
Epoch  4, Batch 416 -Loss:   562.0936 Validation Accuracy: 0.871094
Epoch  4, Batch 417 -Loss:   568.2855 Validation Accuracy: 0.871094
Epoch  4, Batch 418 -Loss:   409.6503 Validation Accuracy: 0.867188
Epoch  4, Batch 419 -Loss:   597.0940 Validation Accuracy: 0.871094
Epoch  4, Batch 420 -Loss:   779.5117 Validation Accuracy: 0.871094
Epoch  4, Batch 421 -Loss:   563.1936 Validation Accuracy: 0.871094
Epoch  4, Batch 422 -Loss:   674.0345 Validation

Epoch  5, Batch 100 -Loss:   440.3391 Validation Accuracy: 0.875000
Epoch  5, Batch 101 -Loss:   453.8288 Validation Accuracy: 0.871094
Epoch  5, Batch 102 -Loss:   721.8612 Validation Accuracy: 0.867188
Epoch  5, Batch 103 -Loss:   312.2575 Validation Accuracy: 0.875000
Epoch  5, Batch 104 -Loss:   617.5355 Validation Accuracy: 0.871094
Epoch  5, Batch 105 -Loss:   533.1194 Validation Accuracy: 0.871094
Epoch  5, Batch 106 -Loss:   536.2062 Validation Accuracy: 0.871094
Epoch  5, Batch 107 -Loss:   250.2982 Validation Accuracy: 0.875000
Epoch  5, Batch 108 -Loss:   537.0225 Validation Accuracy: 0.875000
Epoch  5, Batch 109 -Loss:   699.9020 Validation Accuracy: 0.875000
Epoch  5, Batch 110 -Loss:   345.7417 Validation Accuracy: 0.871094
Epoch  5, Batch 111 -Loss:   671.1543 Validation Accuracy: 0.871094
Epoch  5, Batch 112 -Loss:   266.0201 Validation Accuracy: 0.875000
Epoch  5, Batch 113 -Loss:   655.2521 Validation Accuracy: 0.871094
Epoch  5, Batch 114 -Loss:   751.3489 Validation

KeyboardInterrupt: 

In [17]:
import numpy as py
a = np.array([1,2,3])
print(a.shape)
b = np.array([[1,2,3],[2,3,4]])
print(b.shape)
print(np.dot(a,b.T))

(3,)
(2, 3)
[14 20]


### max_pool

In [1]:
"""
Set the values to `strides` and `ksize` such that
the output shape after pooling is (1, 2, 2, 1).
"""
import tensorflow as tf
import numpy as np

# `tf.nn.max_pool` requires the input be 4D (batch_size, height, width, depth)
# (1, 4, 4, 1)
x = np.array([
    [0, 1, 0.5, 10],
    [2, 2.5, 1, -8],
    [4, 0, 5, 6],
    [15, 1, 2, 3]], dtype=np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)

def maxpool(input):
    # TODO: Set the ksize (filter size) for each dimension (batch_size, height, width, depth)
    ksize = [1, 2, 2, 1]
    # TODO: Set the stride for each dimension (batch_size, height, width, depth)
    strides = [1, 2, 2, 1]
    # TODO: set the padding, either 'VALID' or 'SAME'.
    padding = 'SAME'
    # https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#max_pool
    return tf.nn.max_pool(input, ksize, strides, padding)
    
out = maxpool(X)